<a href="https://colab.research.google.com/github/ankit-rathi/Quantvesting_v2/blob/main/myStocks_Portfolio_Analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install pyxirr
!pip install ta
!pip install yfinance==0.2.59

  Installing build dependencies ... -

 \

 done


  Getting requirements to build wheel ... -

 done


...

 done
  Created wheel for ta: filename=ta-0.11.0-py3-none-any.whl size=29482 sha256=f937a2bb8e4db8d6151cc5ccb92476f13cd39a82daf83f3a28c579fa2a05017a
  Stored in directory: /home/runner/.cache/pip/wheels/ec/b6/81/d781ff040ee9432384560f29596ee14ca79d70529d4228e50f
Successfully built ta


  Attempting uninstall: protobuf
    Found existing installation: protobuf 6.33.2
    Uninstalling protobuf-6.33.2:
      Successfully uninstalled protobuf-6.33.2


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 0/2 [protobuf]

  Attempting uninstall: yfinance
    Found existing installation: yfinance 0.2.66
    Uninstalling yfinance-0.2.66:
      Successfully uninstalled yfinance-0.2.66
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 0/2 [protobuf]

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2/2 [yfinance]


In [2]:
# import libraries

import numpy as np
import pandas as pd
import datetime
import warnings
warnings.filterwarnings('ignore')
import yfinance as yfin
import ta
import matplotlib.pyplot as plt
import requests

In [3]:
# notebook start time
import time
import datetime
import pytz

start_time = time.time()

# function to print date timestamp
def print_date_time():
  tz_NY = pytz.timezone('Asia/Kolkata')
  datetime_NY = datetime.datetime.now(tz_NY)
  print("Run date time (IST):", datetime_NY.strftime("%Y-%m-%d %H:%M:%S"))

In [4]:
# function to get booked and reserved amount
def get_amt():
  # fetch the JSON data from the URL
  url = "https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myPortfolioAmts.json"
  response = requests.get(url)
  pf_amts = response.json()  # parse the JSON data

  # extract values from the JSON
  py_booked_amt_dm = pf_amts["py_booked_amt_dm"]
  py_booked_amt_sv = pf_amts["py_booked_amt_sv"]
  cy_booked_amt_dm = pf_amts["cy_booked_amt_dm"]
  cy_booked_amt_sv = pf_amts["cy_booked_amt_sv"]
  reserve_amt_dm = pf_amts["reserve_amt_dm"]
  reserve_amt_sv = pf_amts["reserve_amt_sv"]

  # perform calculations
  py_booked_amt = py_booked_amt_dm + py_booked_amt_sv
  cy_booked_amt = cy_booked_amt_dm + cy_booked_amt_sv
  reserve_amt = reserve_amt_dm + reserve_amt_sv
  total_booked_amt = py_booked_amt + cy_booked_amt

  return total_booked_amt, reserve_amt, cy_booked_amt, py_booked_amt

gen_output = 0

In [5]:
# function to format the amount
def format_amt(number):
    abs_number = abs(number)

    if abs_number >= 1_00_00_000:  # Crores
        formatted_amt = f"{number / 1_00_00_000:.2f} C"
    elif abs_number >= 1_00_000:  # Lakhs
        formatted_amt = f"{number / 1_00_000:.2f} L"
    elif abs_number >= 1_000:  # Thousands
        formatted_amt = f"{number / 1_000:.2f} K"
    else:
        formatted_amt = f"{number:.2f}"

    return formatted_amt

# function to set start and end date
def get_start_end_date():
  start_date = (datetime.date.today() + datetime.timedelta(days=-365)).strftime('%Y-%m-%d')
  end_date = (datetime.date.today() + datetime.timedelta(days=1)).strftime('%Y-%m-%d')
  return start_date, end_date

# function to fetch my portfolio csv
def get_mypfs_df():
  mypfs_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myPortfolioStocks.csv')
  return mypfs_df

# function to fetch my prospects csv
def get_mypps_df():
  mypps_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myProspectsScrips.csv')
  return mypps_df

# function to fetch screener data
def get_myscreen_df():
  myscreen_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myScreenerDB.csv')
  return myscreen_df

# function to fetch momentum data
def get_myinvmt_df():
  myinvmt_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myInvestments.csv')
  return myinvmt_df

# function to get the stock ids
def get_stock_ids(df_pf):
  stock_n100 = df_pf['Symbol'].unique()

  exclude = ['CADILAHC','MMTC', 'MASFIN']

  stock_ids = df_pf[~df_pf['Symbol'].isin(exclude) ]['Symbol'].unique()

  #mypf = mypf[mypf.Forecast.notnull()]
  #stock_ids = mypf['Symbol'].unique()

  stock_ids.sort()
  return stock_ids

# get features from screener data
def get_screener_features():
  myscreen_df = get_myscreen_df()
  cols = ['Symbol', 'EPS', 'MedPE', 'ROCE%', 'ROE%', 'CapType']
  return myscreen_df[cols]

# get relative strength
def get_relative_strength(stock_list):

    # Dictionary to store stock tickers and their corresponding percentage price change
    stock_changes = {}

    # Loop through each stock and fetch its price data
    for stock in stock_list:
        try:
            # Download the stock data for the given date range
            stock_data = yfin.Ticker(stock + '.NS').history(period='1mo', interval='1d')[map(str.title, ['open', 'close', 'low', 'high', 'volume'])]

            # Calculate the percentage change for the stock
            if len(stock_data) > 0:
                start_price = stock_data['Close'].iloc[0]
                end_price = stock_data['Close'].iloc[-1]
                percent_change = round(((end_price - start_price) / start_price) * 100, 2)
                stock_changes[stock] = percent_change
            else:
                stock_changes[stock] = np.nan  # If no data is available, set to NaN

        except Exception as e:
            print(f"Error fetching data for {stock}: {e}")
            stock_changes[stock] = np.nan

    # Create a DataFrame with stock tickers and their percentage changes
    df = pd.DataFrame(list(stock_changes.items()), columns=['Symbol', 'Percent_Change'])

    # Drop any stocks with missing data (NaN values)
    df = df.dropna()

    # Calculate the percentile rank based on percentage change
    df['RSP'] = round(df['Percent_Change'].rank(pct=True) * 100, 2)
    cols = ['Symbol', 'RSP']
    df = df[cols]
    # Sort by percentile rank (optional)
    df = df.sort_values(by='RSP', ascending=False).reset_index(drop=True)

    return df


In [6]:
# function to get stock technicals
def stock_prec_dev(stock_symbol):
    #stock_symbol = 'ULTRACEMCO.NS'
    short_window = 20
    mid_window = 50
    long_window = 200
    moving_avg = 'SMA'

    start = datetime.datetime(*map(int, start_date.split('-')))
    end = datetime.datetime(*map(int, end_date.split('-')))

    stock_df = yfin.Ticker(stock_symbol + '.NS').history(period='max', interval='1d')[map(str.title, ['open', 'close', 'low', 'high', 'volume'])]

    stock_df['Max'] = round(max(stock_df['Close']),2)
    stock_df = stock_df[(stock_df.index <= end_date) & (stock_df.index >= start_date)]
    stock_df['200_SMA'] = round(stock_df['Close'].rolling(window = 200, min_periods = 1).mean(),0)
    stock_df['Dev%_200'] = round((stock_df['Close'] - stock_df['200_SMA'])*100/stock_df['200_SMA'],2)
    stock_df.dropna(axis = 0, inplace = True) # remove any null rows

    stock_df['20_SMA'] = stock_df['Close'].rolling(window=20).mean()
    stock_df['50_SMA'] = stock_df['Close'].rolling(window=50).mean()
    stock_df['Symbol'] = stock_symbol

    stock_df['Close'] = round(stock_df['Close'],2)
    stock_df['Min'] = round(min(stock_df['Close']),2)
    stock_df['RSI_14'] = ta.momentum.RSIIndicator(close=stock_df['Close'], window=14).rsi()
    stock_df['RSI_14'] = round(stock_df['RSI_14'],0)
    stock_df['Prev_Close'] = stock_df['Close'].shift(1)
    stock_df.drop(['Open', 'Low', 'High', 'Volume'], axis=1, inplace=True)
    stock_df = stock_df.tail(1)

    max_SMA = max(stock_df['20_SMA'].item(), stock_df['50_SMA'].item(), stock_df['200_SMA'].item())
    min_SMA = min(stock_df['20_SMA'].item(), stock_df['50_SMA'].item(), stock_df['200_SMA'].item())
    ABS_Spread = max_SMA - min_SMA
    stock_df['Spread%'] = round((ABS_Spread / stock_df['200_SMA'].item()) * 100,2)

    return stock_df

# function to compute stock attributes
def get_common_features(stock_ids, df_mypf):

  df_prec_dev = pd.DataFrame()
  df_tmp = get_screener_features()
  df_rs = get_relative_strength(stock_ids)

  for stock_id in stock_ids:
      tmp = stock_prec_dev(stock_id)
      tmp = tmp.reset_index()
      df_prec_dev = pd.concat([df_prec_dev, tmp], ignore_index = True)
  df_prec_dev = pd.merge(df_prec_dev, df_mypf, on= 'Symbol')
  df_prec_dev = pd.merge(df_prec_dev, df_tmp, on= 'Symbol', how='left')
  df_prec_dev = pd.merge(df_prec_dev, df_rs, on= 'Symbol', how='left')
  #print(df_prec_dev.columns)
  df_prec_dev['Curr_PE'] = round(df_prec_dev['Close']/df_prec_dev['EPS'],1)
  df_prec_dev['Dev%_PE'] = round((df_prec_dev['Curr_PE'] - df_prec_dev['MedPE'])*100/df_prec_dev['MedPE'],2)
  df_prec_dev['Conviction'] = df_prec_dev['Conviction'] + '-' + df_prec_dev['CapType']
  return df_prec_dev

# function to arrange stock features
def arrange_features(df_stocks, common_cols, diff_cols):
  df_stocks_common = df_stocks[common_cols].drop_duplicates()
  df_stocks_diff = df_stocks[diff_cols]
  df_stocks_diff['Investment'] = df_stocks_diff['AvgCost'] * df_stocks_diff['Shares']
  df_stocks_diff = df_stocks_diff.groupby(['Symbol'])[['Shares', 'Investment']].aggregate(['sum']).reset_index()
  df_stocks_diff.columns = ['Symbol', 'Shares', 'Investment']
  df_stocks_diff['AvgCost'] = round(df_stocks_diff['Investment']/df_stocks_diff['Shares'],2)
  df_stocks = pd.merge(df_stocks_diff, df_stocks_common, on='Symbol')
  return df_stocks

# plot fact distribution across dimension
def plot_pie_chart(df, dimension, fact):
  # grouping the data by category and calculating the sum of fact for each type
  grouped_data = df.groupby(dimension)[fact].sum()

  # sorting the grouped data in descending order
  grouped_data = grouped_data.sort_values(ascending=False)

  # creating a pie chart
  grouped_data.plot.pie(autopct='%1.1f%%', startangle=90, figsize=(6, 6))

  # adding a title and displaying the plot
  plt.title(f'{dimension} {fact} Distribution')
  plt.ylabel('')  # To hide the y-label
  plt.show()

In [7]:
# function to get portfolio features

def get_portfolio_features(df_common_features):

  print_date_time()
  print('-------------------')

  #df_common_features["Target"] = df_common_features["Target"].fillna(df_common_features["Max"])
  df_common_features['NTT'] = np.where(df_common_features['Strategy']== 'NTT', df_common_features["Target"], df_common_features['Max'])
  df_common_features['LTT'] = np.where(df_common_features['Strategy']== 'BTT', df_common_features["Target"], df_common_features['Max'])
  df_common_features['BOL'] = df_common_features['Min']

  tmp_df = df_common_features[df_common_features['Symbol'].isin(mypf_df[mypf_df['InPortfolio'] != 'NA'].Symbol.values)]
  print('qualified stocks: '+str(len(tmp_df['Symbol'].unique())))
  tmp_df1 = tmp_df[tmp_df['LatestQtr'] == 1]
  print('with latest results: '+str(len(tmp_df1['Symbol'].unique())))
  tmp_df1 = tmp_df1[tmp_df1['StarStock'] == 1]
  print('still star stocks: '+str(len(tmp_df1['Symbol'].unique())))
  tmp_df['Investment'] = tmp_df['AvgCost'] * tmp_df['Shares']
  tmp_df['Current'] = round(tmp_df['Close'] * tmp_df['Shares'],0)
  tmp_df['Previous'] = tmp_df['Prev_Close'] * tmp_df['Shares']
  tmp_df['EstimatedST'] = tmp_df['NTT'] * tmp_df['Shares']
  tmp_df['EstimatedLT'] = tmp_df['LTT'] * tmp_df['Shares']
  tmp_df['Current P/L'] = round((tmp_df['Current'] - tmp_df['Investment']),0)
  tmp_df['Today P/L%'] = round((tmp_df['Current'] - tmp_df['Previous'])*100/tmp_df['Previous'],2)
  tmp_df['Current P/L%'] = round((tmp_df['Current'] - tmp_df['Investment'])*100/tmp_df['Investment'],2)
  tmp_df['EstimatedST P/L%'] = round((tmp_df['EstimatedST'] - tmp_df['Investment'])*100/tmp_df['Investment'],2)
  tmp_df['EstimatedLT P/L%'] = round((tmp_df['EstimatedLT'] - tmp_df['Investment'])*100/tmp_df['Investment'],2)
  tmp_df['NTT%'] = round((tmp_df['NTT'] - tmp_df['Close'])*100/tmp_df['Close'],2)
  tmp_df['LTT%'] = round((tmp_df['LTT'] - tmp_df['Close'])*100/tmp_df['Close'],2)
  tmp_df['Gained%'] = round((tmp_df['Close'] - tmp_df['BOL'])*100/tmp_df['BOL'],2)
  investment = round(sum(tmp_df['AvgCost']*tmp_df['Shares']),0)
  current = round(sum(tmp_df['Close']*tmp_df['Shares']),0)
  tmp_df['InitAlloc%'] = round(tmp_df['Investment']*100/investment,2)
  tmp_df['CurrAlloc%'] = round(tmp_df['Current']*100/current,2)
  tmp_df['FTT'] = tmp_df['LTT']
  tmp_df.loc[tmp_df['Strategy'] == 'NTT', 'FTT'] = tmp_df['NTT']
  tmp_df['FTT%'] = tmp_df['LTT%']
  tmp_df.loc[tmp_df['Strategy'] == 'NTT', 'FTT%'] = tmp_df['NTT%']
  tmp_df['FTT Amt'] = round(tmp_df['FTT%'] * tmp_df['Current']/100,0)
  tmp_df['OTT%'] = round((tmp_df['FTT'] - tmp_df['AvgCost'])*100/tmp_df['AvgCost'],2)
  tmp_df['RRR Ind'] = round(tmp_df['Current P/L']/tmp_df['FTT Amt'],2)
  tmp_df['Risk Ind'] = round(tmp_df['Current P/L%']*tmp_df['CurrAlloc%'],0)

  return tmp_df

# function to print portfolio features
def print_portfolio_stats(df_portfolio_features, myinvmt_df):
  from pyxirr import xirr

  total_booked_amt, reserve_amt, cy_booked_amt, py_booked_amt = get_amt()

  dates = myinvmt_df['Date'].values
  dates = np.append(dates, datetime.date.today().strftime('%d-%b-%y'))
  investment = myinvmt_df['Investment'].values
  dates= pd.to_datetime(dates)

  current = round(sum(df_portfolio_features['Close']*df_portfolio_features['Shares']),0) + reserve_amt
  investment_xirr = np.append(investment, current)
  cagr = round(xirr(pd.DataFrame({"dates": dates, "amounts": investment_xirr}))*100,2)

  investment = -sum(investment)
  invested = round(sum(df_portfolio_features['AvgCost']*df_portfolio_features['Shares']),0) + reserve_amt
  previous = round(sum(df_portfolio_features['Prev_Close']*df_portfolio_features['Shares']),0) + reserve_amt
  cy_invested = investment + py_booked_amt

  today_pnl_amount = current-previous
  today_pnl_percentage = round((current-previous)*100/previous,2)

  curr_pnl_amount = current-invested
  curr_pnl_percentage = round((curr_pnl_amount)*100/(cy_invested),2)

  cy_pnl_amount = cy_booked_amt + curr_pnl_amount
  cy_pnl_percentage = round((cy_pnl_amount)*100/cy_invested,2)

  overall_pnl_amount = total_booked_amt + curr_pnl_amount
  overall_pnl_percentage = round((overall_pnl_amount)*100/investment,2)

  estimate_st = round(sum(df_portfolio_features['FTT']*df_portfolio_features['Shares']),0)  + reserve_amt
  est_st_pnl_amount = estimate_st-current
  est_st_pnl_percentage = round((est_st_pnl_amount)*100/current,2)

  estimate_lt = round(sum(df_portfolio_features['LTT']*df_portfolio_features['Shares']),0)  + reserve_amt
  est_lt_pnl_amount = estimate_lt-current
  est_lt_pnl_percentage = round((est_lt_pnl_amount)*100/current,2)

  total_profit = round(sum(df_portfolio_features[df_portfolio_features['Current P/L%'] > 0]['Current']) - sum(df_portfolio_features[df_portfolio_features['Current P/L%'] > 0]['Investment']),0)
  total_loss = round(sum(df_portfolio_features[df_portfolio_features['Current P/L%'] < 0]['Current']) - sum(df_portfolio_features[df_portfolio_features['Current P/L%'] < 0]['Investment']),0)

  cy_booked_percentage = round((cy_booked_amt)*100/current,2)
  py_booked_percentage = round((py_booked_amt)*100/investment,2)
  total_booked_percentage = round((total_booked_amt)*100/investment,2)

  print('-------------------')
  print('Initial Investment: ', format_amt(investment))
  print('CY Investment: ', format_amt(cy_invested))
  print('Reserve: ', format_amt(reserve_amt))
  print('Current: ',  format_amt(current))
  print('-------------------')
  print('Today PnL: '+ '{} ({}%)'.format(format_amt(today_pnl_amount), today_pnl_percentage))
  print('Current PnL: '+ '{} ({}%)'.format(format_amt(curr_pnl_amount), curr_pnl_percentage))
  print('CY Booked + Current PnL: '+ '{} ({}%)'.format(format_amt(cy_pnl_amount), cy_pnl_percentage))
  print('-------------------')
  print('Total profit: ', format_amt(total_profit))
  print('Total loss: ', format_amt(total_loss))
  print('-------------------')
  print('Total Booked + Current PnL: '+ '{} ({}%)'.format(format_amt(overall_pnl_amount), overall_pnl_percentage))
  print('Total Booked PnL: '+ '{} ({}%)'.format(format_amt(total_booked_amt), total_booked_percentage))
  print('Curr Year Booked PnL: '+ '{} ({}%)'.format(format_amt(cy_booked_amt), cy_booked_percentage))
  print('Prev Year Booked PnL: '+ '{} ({}%)'.format(format_amt(py_booked_amt), py_booked_percentage))
  print('===================')
  print('Est FTT: ',  format_amt(estimate_st))
  print('Est FTT PnL: '+ '{} ({}%)'.format(format_amt(est_st_pnl_amount), est_st_pnl_percentage))

  print('===================')
  print('Deployed: ', format_amt(investment))

  print('Current: ', format_amt(current))

  print('CAGR/XIRR %: '+'{}%'.format(cagr))

In [8]:
# get start and end date
start_date, end_date = get_start_end_date()

# get portfolio and prospects data
mypfs_df = get_mypfs_df()
mypps_df = get_mypps_df()
myinvmt_df = get_myinvmt_df()

# merge above datasets
mypf_df = pd.merge(mypfs_df, mypps_df, on="Symbol")

# seggregate dm and sv portfolio
dm_pf = mypf_df[mypf_df['InPortfolio'] == 'DM']
sv_pf = mypf_df[mypf_df['InPortfolio'] == 'SV']

# build portfolio stock dataframe
dm_stocks = get_stock_ids(dm_pf)
sv_stocks = get_stock_ids(sv_pf)
df_stocks = pd.concat([dm_pf,sv_pf], ignore_index = True)

# arrange common and diff stock features
common_cols = ['Symbol', 'Target', 'Criteria', 'Strategy', 'CumlRnk', 'LatestQtr', 'StarStock', 'Conviction', 'Category', 'InFolio', 'XIRR', 'MBQ']
diff_cols = ['Symbol', 'AvgCost', 'Shares']
df_stocks = arrange_features(df_stocks, common_cols, diff_cols)

# get common features
stock_ids = df_stocks['Symbol'].values
df_common_features = get_common_features(stock_ids, df_stocks)

df_common_features.reset_index(drop=True, inplace=True)
df_common_features.drop(['Date'], axis=1, inplace=True)
# get and print portfolio features
df_portfolio_features = get_portfolio_features(df_common_features)

print_portfolio_stats(df_portfolio_features, myinvmt_df)

df = df_portfolio_features
#plot_pie_chart(df, 'CapType', 'Current')
list_ox40n = list(df[df['MBQ'].str.contains('OX40N', na=False)]['Symbol'].values)

Run date time (IST): 2025-12-17 11:08:24
-------------------
qualified stocks: 86
with latest results: 25
still star stocks: 15
-------------------
Initial Investment:  1.28 C
CY Investment:  1.55 C
Reserve:  66.76 K
Current:  1.40 C
-------------------
Today PnL: -58.17 K (-0.41%)
Current PnL: -27.34 L (-17.66%)
CY Booked + Current PnL: -13.05 L (-8.43%)
-------------------
Total profit:  1.35 L
Total loss:  -28.68 L
-------------------
Total Booked + Current PnL: 13.93 L (10.9%)
Total Booked PnL: 41.27 L (32.29%)
Curr Year Booked PnL: 14.29 L (10.23%)
Prev Year Booked PnL: 26.98 L (21.11%)
Est FTT:  2.32 C
Est FTT PnL: 91.97 L (65.83%)
Deployed:  1.28 C
Current:  1.40 C
CAGR/XIRR %: 4.94%


In [9]:
# top 5 near their targets
cols = ['Symbol', 'FTT','Today P/L%', 'Current P/L%', 'FTT%', 'OTT%', 'FTT Amt', 'Current P/L', 'Current', 'Dev%_PE', 'RSI_14','Conviction', 'Spread%',  'CumlRnk', 'RRR Ind', 'CurrAlloc%', 'Gained%', 'Criteria', 'Strategy', 'Category']
df_tmp = df_portfolio_features[~df_portfolio_features['Symbol'].isin(['ENRIN','BLUSPRING','DIGITIDE'])]
df_tmp = df_tmp.sort_values(by = 'FTT Amt', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
77,TTKPRESTIG,770.00,-0.24,-18.27,22.47,0.09,18506.0,-18417.0,82360.0,91.25,34.0,M-SC,2.47,253.0,-1.00,0.59,8.20,OX40N,NTT,DURABLES
82,VOLTAS,1530.00,-0.39,8.65,10.17,19.69,21186.0,16578.0,208320.0,-1.91,57.0,X-MC,2.56,78.0,0.78,1.50,16.03,XY25,NTT,AC
50,MASFIN,398.61,-1.01,-3.77,26.82,22.05,25289.0,-3690.0,94290.0,-16.89,50.0,H-SC,6.85,164.0,-0.15,0.68,37.06,XR,ATH,FINANCE
39,INDIGOPNTS,1408.00,-1.74,-14.91,17.54,0.02,26054.0,-26019.0,148540.0,126.56,45.0,M-SC,14.35,234.0,-1.00,1.07,29.20,OX40N,NTT,PAINTS
43,ITC,452.00,0.12,-1.42,12.38,10.78,29328.0,-3422.0,236896.0,-41.15,46.0,X-LC,1.97,5.0,-0.12,1.70,3.74,X40,NTT,FMCG


In [10]:
# top 5 today
df_tmp = df_portfolio_features.sort_values(by = 'Today P/L%', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
63,SFL,1287.00,2.08,-43.13,122.68,26.63,182929.0,-113107.0,149111.0,12.97,34.0,M-SC,11.83,239.0,-0.62,1.07,2.08,XY24,NTT,MISC
13,BSOFT,831.70,1.64,-21.48,92.03,50.79,101638.0,-30213.0,110440.0,0.89,65.0,H-SC,6.45,151.0,-0.30,0.79,28.59,XR,ATH,IT
16,CERA,9475.00,1.05,-29.28,75.19,23.89,93533.0,-51507.0,124396.0,-31.74,44.0,H-SC,10.76,157.0,-0.55,0.89,7.43,OX40N,NTT,CERAMICS
17,COALINDIA,484.83,0.86,-5.35,25.96,19.23,36970.0,-8048.0,142413.0,11.11,57.0,L-LC,1.64,179.0,-0.22,1.02,14.79,XY25,ATH,MINING
57,RAJOOENG,143.10,0.55,-35.37,116.00,39.61,76850.0,-36250.0,66250.0,-56.22,36.0,H-SC,29.35,135.0,-0.47,0.48,4.38,AR,ATH,MISC


In [11]:
# bottom 5 today
df_tmp = df_portfolio_features.sort_values(by = 'Today P/L%', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
18,COFFEEDAY,80.00,-2.48,-42.26,134.26,35.27,88032.0,-47981.0,65568.0,-55.73,44.0,L-SC,10.67,270.0,-0.55,0.47,60.48,XR,NTT,HOTELS
22,DIXON,18931.72,-2.30,-11.33,42.18,26.07,73011.0,-22126.0,173095.0,-56.89,38.0,X-MC,9.66,56.0,-0.30,1.24,7.81,X40N,ATH,IT
48,KPIGREEN,730.66,-2.05,-16.26,75.49,46.95,79204.0,-20377.0,104920.0,-22.59,41.0,H-SC,7.99,154.0,-0.26,0.75,31.44,MH,ATH,POWER
71,TANLA,1943.92,-1.96,-36.95,248.44,119.69,424129.0,-100044.0,170717.0,-36.34,43.0,H-SC,7.96,156.0,-0.24,1.23,34.87,AR,ATH,IT
19,COLPAL,3726.84,-1.87,-19.50,75.79,41.51,160682.0,-51355.0,212010.0,-4.10,39.0,X-MC,7.32,61.0,-0.32,1.52,0.87,XY25,ATH,FMCG


In [12]:
# top 5 to exit based on CumlRnk
df_tmp = df_portfolio_features[~df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[(df_tmp['Current P/L%'] > -1) & (df_tmp['Current P/L%'] < 1)].sort_values(by = 'CumlRnk', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category


In [13]:
# OX40N stocks to exit
df_tmp = df_portfolio_features[(df_portfolio_features['Symbol'].isin(list_ox40n))].sort_values(by = 'Current P/L%', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
25,FINCABLES,1641.55,-1.04,-2.29,112.77,107.91,162693.0,-3374.0,144270.0,-20.55,56.0,M-SC,12.54,216.0,-0.02,1.04,7.57,OX40N,ATH,CABLES
39,INDIGOPNTS,1408.00,-1.74,-14.91,17.54,0.02,26054.0,-26019.0,148540.0,126.56,45.0,M-SC,14.35,234.0,-1.00,1.07,29.20,OX40N,NTT,PAINTS
77,TTKPRESTIG,770.00,-0.24,-18.27,22.47,0.09,18506.0,-18417.0,82360.0,91.25,34.0,M-SC,2.47,253.0,-1.00,0.59,8.20,OX40N,NTT,DURABLES
66,SIS,528.00,-0.20,-24.33,60.24,21.26,50615.0,-27010.0,84022.0,1981.87,49.0,H-SC,5.07,166.0,-0.53,0.60,13.52,OX40N,NTT,MISC
47,KANSAINER,340.00,-0.74,-25.62,52.57,13.47,105437.0,-69102.0,200565.0,-69.48,33.0,H-SC,5.66,158.0,-0.66,1.44,1.98,XY24,NTT,PAINTS


In [14]:
# top 5 to book for rotation
df_tmp = df_portfolio_features[(df_portfolio_features['Current P/L%'] > 20) ].sort_values(by = 'CumlRnk', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category


In [15]:
# top 5 to monitor for exit
df_tmp = df_portfolio_features[~df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[(df_tmp['Current P/L%'] > 0) ].sort_values(by = 'Dev%_PE', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
84,WIPRO,420.0,0.49,6.84,61.23,72.27,110851.0,11593.0,181041.0,-8.35,65.0,M-LC,2.6,99.0,0.1,1.3,13.44,XR,NTT,IT


In [16]:
# top 5 to book for rotation from weak conviction
df_tmp = df_portfolio_features[~df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[~df_tmp['Criteria'].isin(['XY25','XY24','X40', 'X40N'])].sort_values(by = 'RRR Ind', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
84,WIPRO,420.00,0.49,6.84,61.23,72.27,110851.0,11593.0,181041.0,-8.35,65.0,M-LC,2.60,99.0,0.10,1.30,13.44,XR,NTT,IT
25,FINCABLES,1641.55,-1.04,-2.29,112.77,107.91,162693.0,-3374.0,144270.0,-20.55,56.0,M-SC,12.54,216.0,-0.02,1.04,7.57,OX40N,ATH,CABLES
38,INDIAMART,4810.62,0.54,-4.03,115.39,106.72,136587.0,-4966.0,118370.0,-52.54,36.0,H-SC,3.29,139.0,-0.04,0.85,18.41,AR,ATH,MISC
37,IEX,219.00,0.25,-4.48,55.60,48.63,106195.0,-8956.0,190998.0,-36.48,47.0,H-SC,14.08,136.0,-0.08,1.37,6.56,XR,NTT,MISC
85,ZYDUSLIFE,1286.17,0.12,-3.46,40.50,35.63,81937.0,-7260.0,202314.0,-17.18,36.0,H-MC,2.86,119.0,-0.09,1.45,12.63,AR,ATH,PHARMA


In [17]:
# top 5 to accumulate based on lowest RSI_14
df_tmp = df_portfolio_features[df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])].sort_values(by = 'RSI_14', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
9,BAJAJHFL,181.50,-0.70,-18.82,91.52,55.47,180753.0,-45785.0,197501.0,-28.03,23.0,X-MC,13.42,64.0,-0.25,1.42,1.20,X40N,ATH,FINANCE
53,PAGEIND,51605.09,-0.54,-10.23,42.71,28.11,77220.0,-20610.0,180800.0,-34.99,26.0,X-MC,13.20,55.0,-0.27,1.30,0.00,X40,ATH,APPARELS
11,BATAINDIA,2096.00,-1.02,-39.41,123.16,35.20,96013.0,-50712.0,77958.0,2.24,29.0,X-SC,16.29,92.0,-0.53,0.56,0.00,X40,NTT,FOOTWEAR
8,AWL,485.00,-0.72,-23.62,100.95,53.50,232679.0,-71262.0,230489.0,-62.99,29.0,X-MC,3.28,58.0,-0.31,1.66,2.38,XY24,NTT,FMCG
51,MEDANTA,1486.00,-0.43,-5.23,30.93,24.08,37212.0,-6640.0,120310.0,-12.87,31.0,X-SC,5.18,89.0,-0.18,0.87,13.36,XY24,NTT,HEALTHCARE


In [18]:
# top 5 to accumulate based on Spread and CumlRnk
df_tmp = df_portfolio_features[df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[(df_tmp['CumlRnk'] < 100)].sort_values(by = 'Spread%', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
15,CAMS,950.0,-0.94,-2.16,27.23,24.48,67098.0,-5445.0,246411.0,-81.32,41.0,X-SC,0.25,86.0,-0.08,1.77,20.48,X40N,NTT,MISC
43,ITC,452.0,0.12,-1.42,12.38,10.78,29328.0,-3422.0,236896.0,-41.15,46.0,X-LC,1.97,5.0,-0.12,1.70,3.74,X40,NTT,FMCG
20,DABUR,735.0,0.02,-2.77,47.80,43.70,115289.0,-6878.0,241190.0,-10.70,38.0,X-MC,2.29,72.0,-0.06,1.73,11.86,XY24,BTT,FMCG
33,HINDUNILVR,2922.0,0.12,-11.16,27.94,13.67,63809.0,-28683.0,228380.0,-17.78,32.0,X-LC,2.30,14.0,-0.45,1.64,7.82,XY25,NTT,FMCG
12,BERGEPAINT,680.0,-1.06,-4.81,27.26,21.14,58994.0,-10927.0,216412.0,-11.49,37.0,X-MC,2.36,74.0,-0.19,1.56,21.98,XY24,NTT,PAINTS


In [19]:
# top 5 to accumulate based on lowest Gained%
df_tmp = df_portfolio_features[df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])].sort_values(by = 'Gained%', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
3,ACC,3906.00,-0.08,-25.62,120.69,64.15,213609.0,-60961.0,176990.0,-56.31,34.0,X-SC,2.78,82.0,-0.29,1.27,0.00,XY24,BTT,CEMENT
11,BATAINDIA,2096.00,-1.02,-39.41,123.16,35.20,96013.0,-50712.0,77958.0,2.24,29.0,X-SC,16.29,92.0,-0.53,0.56,0.00,X40,NTT,FOOTWEAR
53,PAGEIND,51605.09,-0.54,-10.23,42.71,28.11,77220.0,-20610.0,180800.0,-34.99,26.0,X-MC,13.20,55.0,-0.27,1.30,0.00,X40,ATH,APPARELS
59,RELAXO,1176.00,-0.20,-48.17,193.85,52.31,145849.0,-69922.0,75238.0,-44.83,39.0,X-SC,7.10,91.0,-0.48,0.54,0.71,X40N,NTT,FOOTWEAR
31,HAVELLS,2069.16,-0.85,-10.36,47.80,32.48,140532.0,-33991.0,294000.0,-14.91,39.0,X-MC,6.10,67.0,-0.24,2.11,0.73,X40,ATH,ELECTRICAL


In [20]:
# top 5 to accumulate based on lowest CurrAlloc%
df_tmp = df_portfolio_features[df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])].sort_values(by = 'CurrAlloc%', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
55,QUESS,424.00,-0.86,-29.99,104.02,42.84,47343.0,-19493.0,45513.0,-54.27,41.0,X-SC,35.36,83.0,-0.41,0.33,2.97,XY24,NTT,MISC
59,RELAXO,1176.00,-0.20,-48.17,193.85,52.31,145849.0,-69922.0,75238.0,-44.83,39.0,X-SC,7.10,91.0,-0.48,0.54,0.71,X40N,NTT,FOOTWEAR
11,BATAINDIA,2096.00,-1.02,-39.41,123.16,35.20,96013.0,-50712.0,77958.0,2.24,29.0,X-SC,16.29,92.0,-0.53,0.56,0.00,X40,NTT,FOOTWEAR
51,MEDANTA,1486.00,-0.43,-5.23,30.93,24.08,37212.0,-6640.0,120310.0,-12.87,31.0,X-SC,5.18,89.0,-0.18,0.87,13.36,XY24,NTT,HEALTHCARE
34,HONAUT,58357.33,-0.99,-17.30,73.30,43.32,98735.0,-28172.0,134700.0,-28.92,34.0,X-SC,4.90,90.0,-0.29,0.97,3.42,X40N,ATH,ELECTRICAL


In [21]:
# top 5 to accumulate based on CumlRnk
df_tmp = df_portfolio_features.sort_values(by = 'CumlRnk', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
73,TCS,4389.96,0.41,-11.52,36.41,20.71,121858.0,-43555.0,334682.0,-23.62,61.0,X-LC,4.24,1.0,-0.36,2.41,11.83,X40,ATH,IT
41,INFY,1972.00,0.32,8.78,23.40,34.24,79648.0,27486.0,340374.0,-15.29,60.0,X-LC,4.26,2.0,0.35,2.45,17.79,X40,NTT,IT
75,TMPV,600.00,0.35,-28.95,73.09,22.98,117309.0,-65399.0,160499.0,-26.74,31.0,X-LC,6.74,3.0,-0.56,1.15,0.95,XY24,NTT,AUTO
81,VBL,671.64,-0.54,-4.14,41.53,35.67,125737.0,-13079.0,302763.0,-15.61,53.0,X-LC,3.99,4.0,-0.10,2.18,9.14,X40N,ATH,FMCG
43,ITC,452.00,0.12,-1.42,12.38,10.78,29328.0,-3422.0,236896.0,-41.15,46.0,X-LC,1.97,5.0,-0.12,1.70,3.74,X40,NTT,FMCG


In [22]:
# top 5 for average up
df_tmp = df_portfolio_features[(df_portfolio_features['Dev%_200'] > 5)].sort_values(by = 'CurrAlloc%', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
6,ASIANTILES,137.00,-0.29,-17.05,119.02,81.67,92612.0,-15998.0,77812.0,6994.44,45.0,L-SC,12.03,271.0,-0.17,0.56,50.69,XR,NTT,CERAMICS
50,MASFIN,398.61,-1.01,-3.77,26.82,22.05,25289.0,-3690.0,94290.0,-16.89,50.0,H-SC,6.85,164.0,-0.15,0.68,37.06,XR,ATH,FINANCE
13,BSOFT,831.70,1.64,-21.48,92.03,50.79,101638.0,-30213.0,110440.0,0.89,65.0,H-SC,6.45,151.0,-0.30,0.79,28.59,XR,ATH,IT
39,INDIGOPNTS,1408.00,-1.74,-14.91,17.54,0.02,26054.0,-26019.0,148540.0,126.56,45.0,M-SC,14.35,234.0,-1.00,1.07,29.20,OX40N,NTT,PAINTS
5,ASIANPAINT,3460.25,0.01,-2.44,23.97,20.95,58877.0,-6142.0,245626.0,-3.44,49.0,X-LC,17.13,31.0,-0.10,1.77,32.33,X40,ATH,PAINTS


In [23]:
# top 5 RSP
df_tmp = df_portfolio_features.sort_values(by = 'RSP', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
13,BSOFT,831.70,1.64,-21.48,92.03,50.79,101638.0,-30213.0,110440.0,0.89,65.0,H-SC,6.45,151.0,-0.30,0.79,28.59,XR,ATH,IT
84,WIPRO,420.00,0.49,6.84,61.23,72.27,110851.0,11593.0,181041.0,-8.35,65.0,M-LC,2.60,99.0,0.10,1.30,13.44,XR,NTT,IT
41,INFY,1972.00,0.32,8.78,23.40,34.24,79648.0,27486.0,340374.0,-15.29,60.0,X-LC,4.26,2.0,0.35,2.45,17.79,X40,NTT,IT
73,TCS,4389.96,0.41,-11.52,36.41,20.71,121858.0,-43555.0,334682.0,-23.62,61.0,X-LC,4.24,1.0,-0.36,2.41,11.83,X40,ATH,IT
27,GLAXO,3466.20,-0.29,3.49,34.45,39.15,71050.0,6956.0,206240.0,-16.92,52.0,X-MC,10.63,60.0,0.10,1.48,33.50,X40,ATH,PHARMA


In [24]:
# Top N allocation
df_tmp = df_portfolio_features.sort_values(by = 'CurrAlloc%', ascending=False)
top_n_values = [10, 25, 50]

sum_df = pd.DataFrame({
    'Top_N': top_n_values,
    'Sum_Alloc%': [df_tmp['CurrAlloc%'].head(n).sum() for n in top_n_values]
})

sum_df

,Top_N,Sum_Alloc%
0,10,20.31
1,25,44.87
2,50,76.65


In [25]:
# market-cap-wise allocation
df_tmp = df_portfolio_features[cols]
df_tmp.groupby(df_tmp['Conviction'].str[-2:])['CurrAlloc%'].sum().sort_values(ascending=False)

Conviction
SC    44.88
MC    30.90
LC    24.17
Name: CurrAlloc%, dtype: float64

In [26]:
# criteria-wise allocation
df_portfolio_features.groupby('Criteria')['CurrAlloc%'].sum().sort_values(ascending=False)

Criteria
XY24     26.70
X40      23.03
X40N     13.03
XR        9.75
XY25      9.08
AR        9.04
OX40N     7.58
SR        0.99
MH        0.75
Name: CurrAlloc%, dtype: float64

In [27]:
# conviction-wise allocation
df_portfolio_features.groupby('Conviction')['CurrAlloc%'].sum().sort_values(ascending=False)

Conviction
X-MC    24.42
H-SC    23.75
X-LC    20.72
M-SC    11.68
X-SC     8.05
H-MC     4.82
L-SC     1.40
M-MC     1.32
M-LC     1.30
H-LC     1.13
L-LC     1.02
L-MC     0.34
Name: CurrAlloc%, dtype: float64

In [28]:
# sector-wise stats
df_tmp = df_portfolio_features.groupby('Category')[['CurrAlloc%', 'Current', 'Current P/L', 'FTT Amt']].sum().sort_values(by=['Current', 'Current P/L'], ascending=False)
df_tmp['Current P/L%'] = round(df_tmp['Current P/L'] * 100 / df_tmp['Current'], 2)
df_tmp['FTT%'] = round(df_tmp['FTT Amt'] * 100 / df_tmp['Current'], 2)
cols = ['CurrAlloc%', 'Current P/L%', 'FTT%']
df_tmp[cols].sort_values(by=['CurrAlloc%'], ascending=False)

,CurrAlloc%,Current P/L%,FTT%
Category,,,
FMCG,17.38,-7.86,42.54
IT,13.03,-15.96,74.26
FINANCE,9.44,-21.39,74.18
MISC,7.28,-28.17,80.60
ELECTRICAL,6.01,-12.24,53.36
PAINTS,5.84,-13.83,30.74
INSURANCE,4.76,-2.28,37.28
PHARMA,4.13,-2.11,34.25
AUTO,2.82,-33.46,79.73


In [29]:
# money to be made criteria-wise
df_portfolio_features.groupby('Criteria')['FTT Amt'].agg(['sum', 'count']).sort_values(by='sum', ascending=False)

,sum,count
Criteria,,
XY24,3210181.0,21
AR,1314245.0,10
XR,1305715.0,13
X40,1019616.0,14
X40N,873073.0,9
OX40N,735929.0,10
XY25,376537.0,6
SR,282615.0,2
MH,79204.0,1


In [30]:
# money to be made conviction-wise
df_portfolio_features.groupby('Conviction')['FTT Amt'].agg(['sum', 'count']).sort_values(by='sum', ascending=False)

,sum,count
Conviction,,
H-SC,3673814.0,25
M-SC,1434914.0,15
X-MC,1428473.0,16
X-LC,844514.0,11
X-SC,800520.0,8
H-MC,408327.0,3
L-SC,266014.0,3
M-LC,110851.0,1
H-LC,76329.0,1


In [31]:
# money to be made criteria and conviction-wise
df_portfolio_features.groupby(['Conviction', 'Criteria'])['FTT Amt'].agg(['sum', 'count']).sort_values(by='sum', ascending=False)

sum  count
Conviction Criteria                  
H-SC       XY24      1287449.0      6
           AR         898697.0      5
M-SC       XY24       824130.0      6
H-SC       XR         784820.0      7
X-MC       X40        497678.0      7
           XY24       406962.0      3
           X40N       341965.0      4
H-SC       OX40N      341029.0      4
X-LC       X40        331264.0      5
X-SC       X40N       311682.0      3
M-SC       OX40N      309530.0      5
X-SC       XY24       298164.0      3
H-SC       SR         282615.0      2
X-LC       X40N       219426.0      2
H-MC       AR         212516.0      2
X-LC       XY24       197665.0      2
H-MC       XY24       195811.0      1
X-SC       X40        190674.0      2
X-MC       XY25       181868.0      2
L-SC       XR         180644.0      2
M-SC       XR         174551.0      2
           AR         126703.0      2
M-LC       XR         110851.0      1
X-LC       XY25        96159.0      2
L-SC       OX40N       85370.0      1
H-SC       MH          79204.0      1
H-LC       AR          76329.0      1
M-MC       XY25        61540.0      1
L-MC       XR          54849.0      1
L-LC       XY25        36970.0      1

In [32]:
# notebook execution time

end_time = time.time()
execution_time = round(end_time - start_time, 0)
print(f"Notebook execution time: {execution_time} seconds")

Notebook execution time: 31.0 seconds
